# Spatial Coherence and Change of Basis
stough, 202-

- [Khan Academy on Change of Basis](https://www.khanacademy.org/math/linear-algebra/alternate-bases/change-of-basis/v/linear-algebra-coordinates-with-respect-to-a-basis)

Complicated way of saying it: Image compression relies on spatial coherence of image data. The fact that a pixel's value is highly correlated with its neighbors most of the time, means that changes of basis like DCT can lead to a much more compressible coordinate representation (more zeros, or minimum description length)...

Little simpler: We are all familiar with plotting points in the plane.

<img src="../dip_figs/xy_plane.png" style="width:150px"/>
<!--![XY-plane](../dip_figs/xy_plane.png)-->

A point $A$ for example might have the coordinates $<4,3>$, by which we mean $4<1,0> + 3<0,1>$, or $4$ in the $x$ direction and $3$ in the $y$. But the same point in space can also be represented as for example $3.5<1,1> + .5<1,-1>$, or rather $<\frac{7}{\sqrt2}, \frac{1}{\sqrt2}>$ if we insist on the two directions we're measuring in ($<1,1>,<1,-1>$) to be unit length. 



If we think about the magnitude of the point, or distance from the origin, notice that in this alternative representation, much of that magnitude ($\frac{7}{8}$th of it in fact) is accounted for by the first component. In fact for a lot of points for which the $x$ and $y$ components are nearly the same, the second component could potentially be ignored without losing too much. This is where the compression will come from: if we can ignore the coefficient, then we don't have to store or transmit it.

In this demo we're going to try to demonstrate how such a change of basis can improve the compressibility of image data. We'll look at a pixel and its neighbor, over all pairs in an image. Spatial coherence implies that most of the time, a pixel and its neighboer will be similar.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)

from vis_utils import (vis_image,
                       vis_pair)

In [ ]:
I = plt.imread('../dip_pics/cat_small.png')
arr_info(I)

In [ ]:
vis_image(I, show_ticks=False, title=None)

In [ ]:
vis_image(I[...,0], show_ticks=False, title=None, cmap='gray')

&nbsp;
## Let's look at pixel pairs
We'll just consider one of the color channels so as not to confuse the issue more.

In [ ]:
pix_pairs = np.reshape(I[...,0], (-1,2))
arr_info(pix_pairs)

In [ ]:
pix_pairs[:10]

In [ ]:
numpoints = 5000
randomInds = np.random.choice(pix_pairs.shape[0], numpoints, replace=False)

In [ ]:
plt.figure()
plt.scatter(pix_pairs[:,0], pix_pairs[:,1], alpha=.01)
plt.axes().set_aspect('equal')

In [ ]:
H = np.array([[1, 1],[1, -1]])/np.sqrt(2)

In [ ]:
H

In [ ]:
new_coords = np.matmul(H, pix_pairs.T).T
arr_info(new_coords)

In [ ]:
np.mean(new_coords, axis=0)

&nbsp;
## Let's see how much energy is in each component for each pair.
We want each pair to sum to 1, where the components are positive. Sort of, what percentage of the total is in each component.

In [ ]:
# We want to use the absolute values, and divide by the sum so that the sum of each pair is one.
# The np.abs(new_coords) will have shape (N,2), whereas the .sum(axis=-1) produces a (N,). 
# In order to broadcast the shapes correctly, we can add [:,None] so that we're dividing 
# (N,2) with (N,1), and numpy will know what that means. Or just keepdims!
normed_coords = np.abs(new_coords) / np.abs(new_coords).sum(axis=-1, keepdims=True)

In [ ]:
normed_coords[:10]

In [ ]:
normed_coords[:10].sum(axis=-1, keepdims=True)

In [ ]:
bins = np.arange(0,1, 1/400)

In [ ]:
f, ax = plt.subplots()
ax.hist(normed_coords[:,0], bins, alpha = .6, label = r'$c_0$', color = 'r');
ax.hist(normed_coords[:,1], bins, alpha = .6, label = r'$c_1$', color = 'b');

plt.legend();

It looks like in the new representation, the $c_0$ component, associated with the $<1,1>$ direction, has almost all the power/energy/magnitude associated with any pair. How does this compare to the original representation?

In [ ]:
normed_pix = np.abs(pix_pairs) / np.abs(pix_pairs).sum(axis=-1, keepdims=True)
bins = np.arange(normed_pix.min(), normed_pix.max(), 1/400)

In [ ]:
f, ax = plt.subplots()
ax.hist(normed_pix[:,0], bins, alpha = .6, label = r'$pix_0$', color = 'r');
ax.hist(normed_pix[:,1], bins, alpha = .6, label = r'$pix_1$', color = 'b');

plt.legend();

The above plot shows that when we consider a pair as two indpendent pixels, then each component is equally important. 

&nbsp;
## What if we zeroed out all of that less important dimension?
Given that the $c_1$ components seem to matter so little, what if we just didn't keep them? Would that change the image all that much? 

In [ ]:
new_coords[:,1] = 0

In [ ]:
rec_pairs = np.matmul(new_coords, H.T)
arr_info(rec_pairs)

In [ ]:
Ir = np.reshape(rec_pairs, I.shape[:2])

In [ ]:
vis_pair(I[...,0], Ir, cmap='gray', show_ticks=False)

### Little experiment figuring out how to consider a pixel and its horizontal neighbor.

In [ ]:
a = np.arange(16).reshape(4,4)

In [ ]:
a

In [ ]:
np.reshape(a, (2,))

In [ ]:
np.reshape(a, (8,2))

In [ ]:
np.reshape(a, (8,-1))

In [ ]:
np.reshape(a, (-1,2))